# Trading Performance Visualization
Visualiserar trading-performance baserat på situation_analysis data.

**Kör först:** `python situation_analysis.py` för att generera `data/annotated_trades.csv`

## Setup & Data Loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Load data
df = pd.read_csv('data/annotated_trades.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['is_win'] = df['is_win'].astype(bool)

print(f"📊 Loaded {len(df)} trades")
print(f"   Period: {df['timestamp'].min()} to {df['timestamp'].max()}")
print(f"   Overall win-rate: {df['is_win'].mean()*100:.1f}%")
df.head()

## 1. Win-Rate by Hour (Heatmap)

In [ ]:
# Group by hour and calculate win-rate
hourly = df.groupby('hour_utc').agg({
    'is_win': ['sum', 'count', 'mean'],
    'pnl_pct': 'mean'
}).round(3)

hourly.columns = ['wins', 'total', 'win_rate', 'avg_pnl']
hourly['win_rate_pct'] = hourly['win_rate'] * 100

# Plot
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Win-rate bar chart
ax1 = axes[0]
bars = ax1.bar(hourly.index, hourly['win_rate_pct'], color=['green' if x > 50 else 'red' for x in hourly['win_rate_pct']])
ax1.axhline(50, color='black', linestyle='--', linewidth=1, label='50% threshold')
ax1.set_xlabel('Hour (UTC)')
ax1.set_ylabel('Win Rate (%)')
ax1.set_title('Win-Rate by Hour of Day', fontsize=14, fontweight='bold')
ax1.set_xticks(range(24))
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# Trade count
ax2 = axes[1]
ax2.bar(hourly.index, hourly['total'], color='steelblue', alpha=0.7)
ax2.set_xlabel('Hour (UTC)')
ax2.set_ylabel('Number of Trades')
ax2.set_title('Trade Volume by Hour', fontsize=14, fontweight='bold')
ax2.set_xticks(range(24))
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📈 Best hours (>50% win-rate):")
best = hourly[hourly['win_rate_pct'] > 50].sort_values('win_rate_pct', ascending=False)
if not best.empty:
    for hour, row in best.iterrows():
        print(f"   Hour {hour:02d}: {row['win_rate_pct']:.1f}% ({row['total']:.0f} trades)")
else:
    print("   None found - need more data or better strategy!")

## 2. Win-Rate by Trade Duration

In [ ]:
# Create duration bins
df['duration_bin'] = pd.cut(df['duration_sec'], 
                             bins=[0, 30, 60, 180, 600, float('inf')],
                             labels=['0-30s', '30-60s', '1-3min', '3-10min', '>10min'])

duration_stats = df.groupby('duration_bin').agg({
    'is_win': ['sum', 'count', 'mean'],
    'pnl_pct': 'mean',
    'mfe_pct': 'mean',
    'mae_pct': 'mean'
}).round(4)

duration_stats.columns = ['wins', 'total', 'win_rate', 'avg_pnl', 'avg_mfe', 'avg_mae']

# Plot
fig, ax = plt.subplots(figsize=(12, 6))
x = range(len(duration_stats))
bars = ax.bar(x, duration_stats['win_rate'] * 100, 
              color=['green' if wr > 0.5 else 'red' for wr in duration_stats['win_rate']])
ax.axhline(50, color='black', linestyle='--', linewidth=1)
ax.set_xticks(x)
ax.set_xticklabels(duration_stats.index)
ax.set_ylabel('Win Rate (%)')
ax.set_title('Win-Rate by Trade Duration', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

# Add count labels on bars
for i, (idx, row) in enumerate(duration_stats.iterrows()):
    ax.text(i, row['win_rate']*100 + 2, f"n={row['total']:.0f}", ha='center', fontsize=10)

plt.tight_layout()
plt.show()

print("\n⏱️ Duration Analysis:")
print(duration_stats[['total', 'win_rate', 'avg_pnl']].to_string())

## 3. Win-Rate by Markov State

In [ ]:
state_stats = df.groupby('state').agg({
    'is_win': ['sum', 'count', 'mean'],
    'pnl_pct': 'mean',
    'duration_sec': 'mean'
}).round(4)

state_stats.columns = ['wins', 'total', 'win_rate', 'avg_pnl', 'avg_duration']

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
colors = {'LW': 'darkgreen', 'LB': 'orange', 'SW': 'green', 'SB': 'red'}
bar_colors = [colors.get(state, 'gray') for state in state_stats.index]

x = range(len(state_stats))
bars = ax.bar(x, state_stats['win_rate'] * 100, color=bar_colors)
ax.axhline(50, color='black', linestyle='--', linewidth=1)
ax.set_xticks(x)
ax.set_xticklabels(state_stats.index)
ax.set_ylabel('Win Rate (%)')
ax.set_title('Win-Rate by Markov State', fontsize=14, fontweight='bold')
ax.set_ylim(0, 105)
ax.grid(axis='y', alpha=0.3)

# Add labels
for i, (idx, row) in enumerate(state_stats.iterrows()):
    ax.text(i, row['win_rate']*100 + 2, 
            f"{row['win_rate']*100:.1f}%\nn={row['total']:.0f}", 
            ha='center', fontsize=10)

plt.tight_layout()
plt.show()

print("\n🎲 State Analysis:")
print(state_stats.to_string())

## 4. PnL Distribution (Win vs Loss)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
ax1 = axes[0]
wins = df[df['is_win']]['pnl_pct'] * 100
losses = df[~df['is_win']]['pnl_pct'] * 100

ax1.hist(wins, bins=20, alpha=0.7, color='green', label=f'Wins (n={len(wins)})')
ax1.hist(losses, bins=20, alpha=0.7, color='red', label=f'Losses (n={len(losses)})')
ax1.axvline(0, color='black', linestyle='--', linewidth=1)
ax1.set_xlabel('PnL (%)')
ax1.set_ylabel('Frequency')
ax1.set_title('PnL Distribution', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(alpha=0.3)

# Cumulative PnL over time
ax2 = axes[1]
df_sorted = df.sort_values('timestamp')
df_sorted['cumulative_pnl'] = df_sorted['pnl_pct'].cumsum() * 100
ax2.plot(df_sorted['timestamp'], df_sorted['cumulative_pnl'], linewidth=2, color='steelblue')
ax2.axhline(0, color='black', linestyle='--', linewidth=1)
ax2.set_xlabel('Time')
ax2.set_ylabel('Cumulative PnL (%)')
ax2.set_title('Cumulative PnL Over Time', fontsize=14, fontweight='bold')
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n💰 PnL Summary:")
print(f"   Total PnL: {df['pnl_pct'].sum()*100:.3f}%")
print(f"   Avg Win: {wins.mean():.3f}% (n={len(wins)})")
print(f"   Avg Loss: {losses.mean():.3f}% (n={len(losses)})")
print(f"   Win/Loss Ratio: {abs(wins.mean() / losses.mean()):.2f}" if len(losses) > 0 and losses.mean() != 0 else "N/A")

## 5. MFE/MAE Analysis

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

# Scatter plot: MAE vs MFE, colored by win/loss
wins = df[df['is_win']]
losses = df[~df['is_win']]

ax.scatter(wins['mae_pct']*100, wins['mfe_pct']*100, 
           c='green', alpha=0.6, s=100, label=f'Wins (n={len(wins)})', edgecolors='black')
ax.scatter(losses['mae_pct']*100, losses['mfe_pct']*100, 
           c='red', alpha=0.6, s=100, label=f'Losses (n={len(losses)})', edgecolors='black')

ax.set_xlabel('MAE - Max Adverse Excursion (%)', fontsize=12)
ax.set_ylabel('MFE - Max Favorable Excursion (%)', fontsize=12)
ax.set_title('MFE vs MAE (Win/Loss Distribution)', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(alpha=0.3)

# Add reference lines
ax.axhline(0.1, color='blue', linestyle='--', alpha=0.5, label='TP level (0.10%)')
ax.axvline(0.08, color='orange', linestyle='--', alpha=0.5, label='Loss threshold (0.08%)')

plt.tight_layout()
plt.show()

print("\n📉 MFE/MAE Insights:")
print(f"   Wins - Avg MFE: {wins['mfe_pct'].mean()*100:.3f}%, Avg MAE: {wins['mae_pct'].mean()*100:.3f}%")
print(f"   Losses - Avg MFE: {losses['mfe_pct'].mean()*100:.3f}%, Avg MAE: {losses['mae_pct'].mean()*100:.3f}%")

## 6. Generate Recommendations

In [ ]:
print("\n" + "="*80)
print("AUTOMATED RECOMMENDATIONS")
print("="*80)

# Overall performance
overall_wr = df['is_win'].mean()
print(f"\n1. Overall Win-Rate: {overall_wr*100:.1f}%")
if overall_wr < 0.4:
    print("   ⚠️ CRITICAL: Win-rate under 40% - major strategy revision needed!")
    print("   → Increase TP_PCT, MIN_MOVE_PCT, COOLDOWN_SEC")
    print("   → Enable time_filter to avoid bad hours")
elif overall_wr < 0.5:
    print("   ⚠️ WARNING: Win-rate under 50% - needs improvement")
    print("   → Fine-tune TP_PCT and entry filters")
else:
    print("   ✅ Good win-rate! Continue monitoring")

# Hour recommendations
print("\n2. Time Filter Recommendations:")
best_hours = hourly[hourly['win_rate_pct'] > 50].index.tolist()
worst_hours = hourly[hourly['win_rate_pct'] < 30].index.tolist()

if best_hours:
    print(f"   ✅ Trade these hours: {best_hours}")
if worst_hours:
    print(f"   ❌ Avoid these hours: {worst_hours}")

# Duration recommendations
print("\n3. Trade Duration Insights:")
quick_wr = duration_stats.loc[duration_stats.index.isin(['0-30s', '30-60s']), 'win_rate'].mean()
long_wr = duration_stats.loc[duration_stats.index.isin(['3-10min', '>10min']), 'win_rate'].mean()

print(f"   Quick trades (<60s): {quick_wr*100:.1f}% win-rate")
print(f"   Long trades (>3min): {long_wr*100:.1f}% win-rate")

if quick_wr < 0.3:
    print("   ⚠️ Quick trades perform poorly!")
    print("   → Increase MIN_MOVE_PCT to 0.0003 or higher")
    print("   → Increase COOLDOWN_SEC to 5-10s")

# State recommendations
print("\n4. State Performance:")
for state in ['LW', 'LB', 'SW', 'SB']:
    if state in state_stats.index:
        wr = state_stats.loc[state, 'win_rate']
        count = state_stats.loc[state, 'total']
        print(f"   {state}: {wr*100:.1f}% ({count:.0f} trades)")

be_ratio = state_stats.loc[state_stats.index.isin(['LB', 'SB']), 'total'].sum() / len(df)
if be_ratio > 0.6:
    print(f"\n   ⚠️ {be_ratio*100:.1f}% of trades are breakeven!")
    print("   → Increase TP_PCT to capture more profit")
    print("   → Consider using trailing stops")

print("\n" + "="*80)
print("✅ Analysis complete! Use these insights to tune your config.json")
print("="*80)